<a href="https://colab.research.google.com/github/aidyai/rss-ai/blob/main/rss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  apify-client langchain-openai langchain
!pip install feedparser newspaper3k openai
#!pip install openai==0.28
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document

import feedparser
import requests
from bs4 import BeautifulSoup
import openai
from newspaper import Article

In [ ]:
import os
os.environ["APIFY_API_TOKEN"] = ""
os.environ["OPENAI_API_KEY"] = ""


## ARTICLE EXTRACTION: This code only extracts onearticle but can be modified to work for the latest 10, 50 or 20 articles

In [ ]:
# RSS feed URL
rss_feed_url = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Get the latest just one articles
latest_articles = feed.entries[8:9]   # we just yse one for speed

In [ ]:
print(latest_articles)

[{'title': 'Akpabio To Eno: You have a divine duty to unite Akwa Ibom People - Champion Newspapers', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen', 'value': 'Akpabio To Eno: You have a divine duty to unite Akwa Ibom People - Champion Newspapers'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5'}], 'link': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5', 'id': 'CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA', 'guidislink': False, 'published': 'Sat, 04 May 2024 16:27:45 GMT', 'published_parsed':

In [ ]:
apify = ApifyWrapper()

In [ ]:
title = []
urls_to_crawl = []

for article in latest_articles:

    # extracting the title and article URL
    title = article.title
    article_url = article.link



    # Appending the article title and URL
    #title.append({"title": title})
    urls_to_crawl.append({"url": article_url})

In [ ]:
print(urls_to_crawl)

[{'url': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5'}]


In [ ]:
#this functions looks into the urls_to_crawl and extracts its text
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": urls_to_crawl},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

# Extracting the title, article_url and article_text for the latest 10 articles

In [ ]:
print(title)                      #EXTRACTED TITLE
print(urls_to_crawl)              #EXTRACTED URL

article_text = loader.load()
article_text                      #EXTRACTED ARTICLE TEXY

Akpabio To Eno: You have a divine duty to unite Akwa Ibom People - Champion Newspapers
[{'url': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5'}]


[Document(page_content='President of the Senate, Godswill Akpabio has charged the Akwa Ibom State Governor, Pastor Umo Eno, on the need to unite the state across party lines for the continued development and progress of the state and not make his father’s enemies his enemies.\nHe described the Governor as a man of peace and an Apostle and disciple of peace, noting, “Akwa Ibom is a very easy state to rule and govern but also a very difficult place to unite.”\nAkpabio gave the charge while on a courtesy visit to the Governor in his office, in Uyo on Friday.\nA statement by his Special Assistant on Media, Jackson Udom, quoted the Senate President as saying, “it is a major duty bestowed on you by God to unite the whole state. Let your actions speak unity to the the state. As a governor, I united the state by appointing people into key positions without considering where they came from.\n“I want you to unite your people without any political considerations. Even if you inherited a policy th

In [ ]:
import json
# Extracting content from the list
extracted_content = [document.page_content for document in article_text]

In [ ]:
extracted_content

['President of the Senate, Godswill Akpabio has charged the Akwa Ibom State Governor, Pastor Umo Eno, on the need to unite the state across party lines for the continued development and progress of the state and not make his father’s enemies his enemies.\nHe described the Governor as a man of peace and an Apostle and disciple of peace, noting, “Akwa Ibom is a very easy state to rule and govern but also a very difficult place to unite.”\nAkpabio gave the charge while on a courtesy visit to the Governor in his office, in Uyo on Friday.\nA statement by his Special Assistant on Media, Jackson Udom, quoted the Senate President as saying, “it is a major duty bestowed on you by God to unite the whole state. Let your actions speak unity to the the state. As a governor, I united the state by appointing people into key positions without considering where they came from.\n“I want you to unite your people without any political considerations. Even if you inherited a policy that was not in the best

In [ ]:
import openai
# Send the content to OpenAI to get topic and category tags
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=extracted_content,
    max_tokens=50,  # Adjust as needed
    n=1,  # Number of completions to generate
    stop=["\n"],  # Stop completion at the end of the text
    temperature=0.5,  # Adjust as needed
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    logit_bias={},
    logprobs=0,
    echo=True  # Print the input prompt along with the completion
)


# USING REPLICATE API TO CALL POWERFUL LARGE LANGUAGE MODELS

In [ ]:
import replicate

In [ ]:
import os
import replicate
from replicate.client import Client

replicate = Client(api_token="")

## USING MISTRAL 8x7B via Replicate Api

In [ ]:
for event in replicate.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSIATAT THAT CAN GENERATE topic tags AND  article category tags",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "prompt_template": "<s>[INST] {prompt} [/INST] ",
        "presence_penalty": 0
    },
):
    print(str(event), end="")

ReplicateError: ReplicateError Details:
title: Free time limit reached
status: 402
detail: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.

## USING METAS LLAMA 3 Via Replicate Api

In [ ]:
for event in replicate.stream(
    "meta/meta-llama-3-70b-instruct",
    input={
        "top_k": 0,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "max_tokens": 650,
        "min_tokens": 0,
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSITANT THAT CAN GENERATE topic tags AND article category tags",
        "length_penalty": 1,
        "stop_sequences": "<|end_of_text|>,<|eot_id|>",
        "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "presence_penalty": 1.15,
        "log_performance_metrics": False
    },
):
    print(str(event), end="")

Here are five topic tags and five category tags for the given article:

**Topic Tags:**

1. Oil Production
2. Nigeria Energy Sector
3. NNPC E&P Ltd
4. Natural Oilfield Services Ltd
5. Oil Mining Lease (OML) 13

**Category Tags:**

1. Business
2. Energy
3. Nigeria News
4. Oil and Gas
5. Economy

## RECIEVES NOTIFICATION

In [ ]:
!pip install replicate

In [ ]:
import replicate
replicate_api_key = ''

In [ ]:
# Set the API key
rep_api = replicate.Client(api_token=replicate_api_key)

In [ ]:
# Define the model and input prompt
model_name = "mistralai/mixtral-8x7b-instruct-v0.1"
input_prompt = "list names of 50 Christian Inventors?"

# Run the model
try:
    iterator = rep_api.run(
        model_name,
        input={"prompt": input_prompt, 'max_length': 500}
    )

    # Retrieve and print the results
    res_text_list = []
    for txt in iterator:
        res_text_list.append(txt)
    output_text = ''.join(res_text_list)
    print(output_text)
except replicate.exceptions.ReplicateError as e:
    print(f"ReplicateError: {e}")

I'm glad to assist you! Here are 50 Christian inventors who have made significant contributions in various fields. Please note that this list is not exhaustive and there are many other Christian inventors who have made important contributions throughout history.

1. Alexander Graham Bell - Inventor of the telephone
2. Tim Berners-Lee - Inventor of the World Wide Web
3. John Logie Baird - Inventor of the mechanical television
4. Guglielmo Marconi - Inventor of wireless telegraphy
5. Joseph Nicéphore


In [ ]:
def ask_user():
    user_input = input("Please enter something: ")
    print(f"You entered: {user_input}")


In [ ]:
# Call the function
ask_user()


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
!pip install -q -U langchain transformers bitsandbytes accelerate langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.2 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [6]:
model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [7]:
device = "cuda" # the device to load the model onto

In [26]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]


encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model_4bit.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s>  [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here's my recipe for a classic mayonnaise:

Ingredients:

* 1 egg
* 1 tablespoon Dijon mustard
* 2 tablespoons apple cider vinegar
* 1/2 teaspoon salt
* 1/4 teaspoon pepper
* 1 cup vegetable oil

Instructions:

1. In a large mixing bowl, whisk together the egg, Dijon mustard, apple cider vinegar, salt, and pepper until well combined.
2. Slowly pour in the vegetable oil while whisking continuously until the mixture thickens.
3. Continue whisking until the mixture becomes thick and creamy, for about 5-10 minutes.
4. Transfer the mixture to an airtight container and refrigerate until ready to use. It will last up to 7 days in the fridge.

That's it! You can use this mayonnaise as a base for your favourite dips and sauces, or

In [8]:
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        do_sample=True,
        top_k=15,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipeline_inst)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [15]:
template = """
[INST] <>
Generate one (1) Topic from this text.
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [20]:
query = "Today I have a meeting with a potential investor. I also have a team meeting where I will be setting the tone for the day/week and ensuring that the team is well aligned"
result = llm(prompt.format(text=query))
result

In [21]:
result

'\n[INST] <>\nGenerate one (1) Topic from this text.\n<>\n\nToday I have a meeting with a potential investor. I also have a team meeting where I will be setting the tone for the day/week and ensuring that the team is well aligned [/INST]\nTopic: Preparation for Meetings with Investors and Team'

In [25]:
template = """
[INST] <>
5 Blog Post Titles Generator from text.
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [26]:
query = "Preparation for Meetings with Investors and Team"
result = llm(prompt.format(text=query))
result

'\n[INST] <>\n5 Blog Post Titles Generator from text.\n<>\n\nPreparation for Meetings with Investors and Team [/INST]\n\n1. "Mastering the Art of Pitching: Tips for Meetings with Investors and Team"\n2. "The Key Elements of a Successful Investor Meeting"\n3. "From Idea to Execution: Preparing for Investor Meetings"\n4. "The Importance of a Strong Investor Presentation"\n5. "Building Strong Relationships with Investors and Team Members"'

In [28]:
template = """
[INST] <>
Content Creator and Writer that generates Long articles.
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [29]:
query = "The Key Elements of a Successful Investor Meeting"
result = llm(prompt.format(text=query))
result

"\n[INST] <>\nContent Creator and Writer that generates Long articles.\n<>\n\nThe Key Elements of a Successful Investor Meeting [/INST]\n\nAs an investor, attending a meeting with potential investors is an exciting and potentially life-changing opportunity. However, it can also be overwhelming and stressful if you're not well prepared. To ensure a successful investor meeting, here are the key elements you need to focus on.\n\n1. Clear and compelling pitch\n\nYour pitch should clearly explain your business, your market, and your competitive advantage. It should capture the investor's attention and convince them that your idea is worth pursuing. Practice your pitch until you can deliver it fluently and confidently.\n\n2. Financial projections\n\nProper financial projections are critical to convince investors that your business has potential for growth. Investors want to see that you have analyzed your revenue, expenses, and cash flow, and that you have realistic financial goals. Be prepa

In [ ]:
# Define the template for generating content
template = """<s>[INST] You are a respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational English.
Answer the question below from context below :
{question} [/INST] </s>
"""

# Travel agency details dictionary
new_travel_agency_details_dict = {
    'brand_creator_user_id': '874nfd827sdf872nsd83274',
    'brand_creator_role': 'ceo_founder',  # other options are 'social_media_manager', 'content_marketer', 'growth_marketer', 'other_employee'
    'brand_id': 'travel12345',
    'brand_name': 'GlobeTrekker',
    'brand_handle': 'globetrekker',
    'brand_image_url': 'https://example.com/images/globetrekker_logo.png',
    'brand_type': 'travel_agency',  # this would be prefilled based on the user's selection at the previous page
    'brand_website_url': 'www.globetrekker.com',
    'brand_description': 'GlobeTrekker is a travel agency that specializes in creating personalized travel experiences. We offer curated travel packages, custom itineraries, and 24/7 customer support to ensure your journey is seamless and unforgettable.',
    'other': {}
}

In [ ]:
template = """<s>[INST] You are an respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational english.
Answer the question below from context below :
{question} [/INST] </s>
"""

def generate_response(question):
  prompt = PromptTemplate(template=template, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question})
  return response

In [18]:
# Define the template for generating a title
title_template = """<s>[INST] Generate a catchy and engaging title based on the following user input:
{user_input} [/INST] </s>"""

# Function to generate title
def generate_title(user_input):
    prompt = PromptTemplate(template=title_template, input_variables=["user_input"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    response = llm_chain.run({"user_input": user_input})
    # Extract title from response
    title = response.strip().replace("<s>", "").replace("</s>", "").split(":")[-1].strip()
    return title

In [20]:
generate_title("i want to talk about the different airlines and the cost and the country they visit")

'i want to talk about the different airlines and the cost and the country they visit [/INST]'

In [ ]:
talking_points_template = """<s>[INST] Generate 5 talking points based on the following user input:
{user_input} [/INST] </s>"""

story_template = """<s>[INST] Generate a detailed story based on the following talking point:
{talking_point} [/INST] </s>"""

# Combined function
def generate_content(user_input):
    # Generate title
    title_prompt = PromptTemplate(template=title_template, input_variables=["user_input"])
    llm_chain = LLMChain(prompt=title_prompt, llm=llm)
    title_response = llm_chain.run({"user_input": user_input})
    title = title_response.strip().replace("<s>", "").replace("</s>", "").split(":")[-1].strip()

    return title

    # Generate talking points
    tp_prompt = PromptTemplate(template=talking_points_template, input_variables=["user_input"])
    llm_chain = LLMChain(prompt=tp_prompt, llm=llm)
    tp_response = llm_chain.run({"user_input": user_input})
    talking_points = [tp.strip() for tp in tp_response.strip().replace("<s>", "").replace("</s>", "").split('\n') if tp.strip()]
    talking_points = talking_points[:5]

    return talking_points
    #print("Generated Talking Points:")
    for i, point in enumerate(talking_points, 1):
        print(f"{i}. {point}")

    # Use one of the talking points to generate a story
    if talking_points:
        chosen_talking_point = talking_points[0]
        story_prompt = PromptTemplate(template=story_template, input_variables=["talking_point"])
        llm_chain = LLMChain(prompt=story_prompt, llm=llm)
        story_response = llm_chain.run({"talking_point": chosen_talking_point})
        story = story_response.strip().replace("<s>", "").replace("</s>", "").split(":")[-1].strip()

        return story


In [ ]:
generate_content("A Comparative Analysis of Airlines and Their Global Reach")

'A Comparative Analysis of Airlines and Their Global Reach [/INST]'

In [ ]:
def generate_response(question, context):
    prompt = PromptTemplate(template=template, input_variables=["question", "context"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    response = llm_chain.run({"question": question, "context": context})
    return response

In [ ]:
template = """<s>[INST] You are an respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational english.
Answer the question below from context below :
{question} [/INST] </s>
"""

def generate_response(question):
  prompt = PromptTemplate(template=template, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question})
  return response

In [ ]:
generate_response("list 16 innovators/inventors that believe in God")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Travel agency details dictionary
new_travel_agency_details_dict = {
    'brand_creator_user_id' : '874nfd827sdf872nsd83274',
    'brand_creator_role' : 'ceo_founder',  # other options are 'social_media_manager', 'content_marketer', 'growth_marketer', 'other_employee'
    'brand_id' : 'travel12345',
    'brand_name' : 'GlobeTrekker',
    'brand_handle' : 'globetrekker',
    'brand_image_url' : 'https://example.com/images/globetrekker_logo.png',
    'brand_type' : 'travel_agency',  # this would be prefilled based on the user's selection at the previous page
    'brand_website_url' : 'www.globetrekker.com',
    'brand_description' : 'GlobeTrekker is a travel agency that specializes in creating personalized travel experiences. We offer curated travel packages, custom itineraries, and 24/7 customer support to ensure your journey is seamless and unforgettable.',
    'other' : {}
}

In [ ]:
i want to talk about the different airlines and the cost and the country they visit

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the tokenizer and model
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set the model to evaluation mode and use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

In [ ]:
# Travel agency details dictionary
new_travel_agency_details_dict = {
    'brand_creator_user_id' : '874nfd827sdf872nsd83274',
    'brand_creator_role' : 'ceo_founder',
    'brand_id' : 'travel12345',
    'brand_name' : 'GlobeTrekker',
    'brand_handle' : 'globetrekker',
    'brand_image_url' : 'https://example.com/images/globetrekker_logo.png',
    'brand_type' : 'travel_agency',  # this would be prefilled based on the user's selection at the previous page
    'brand_website_url' : 'www.globetrekker.com',
    'brand_description' : 'GlobeTrekker is a travel agency that specializes in creating personalized travel experiences. We offer curated travel packages, custom itineraries, and 24/7 customer support to ensure your journey is seamless and unforgettable.',
    'other' : {}
}

def generate_travel_content(travel_agency_details):
    # Extract relevant details to provide context to the AI
    agency_info = (
        f"Brand Name: {travel_agency_details['brand_name']}\n"
        f"Brand Description: {travel_agency_details['brand_description']}\n"
        f"Website: {travel_agency_details['brand_website_url']}\n\n"
    )

    # Ask the travel agency for input
    user_input = input("Please describe the type of travel content you would like to generate: ")

    # Combine the agency info with the user input
    prompt = agency_info + "User Request: " + user_input

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate content
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=150,  # Adjust the max_length as needed
            num_return_sequences=1,
            temperature=0.7,  # Adjust temperature for creativity
            top_k=50,        # Adjust top_k for more/less randomness
            top_p=0.95       # Adjust top_p for more/less randomness
        )

    # Decode and print the generated content
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\nGenerated Travel Content:\n")
    print(generated_text)

# Call the function to generate content with travel agency details
generate_travel_content(new_travel_agency_details_dict)


In [14]:
class TravelContentGenerator:
    def __init__(self):

        self.llm = HuggingFacePipeline(pipeline=pipeline_inst)

        # Define the templates
        self.title_template = """<s>[INST] Generate a catchy and engaging title based on the following user input:
        {user_input} [/INST] </s>"""

        self.talking_points_template = """<s>[INST] Generate 5 talking points based on the following title:
        {title} [/INST] </s>"""

        self.article_template = """<s>[INST] Generate a detailed story based on the following talking point:
        {talking_point} [/INST] </s>"""

    def generate_title(self, user_input):
        # Generate title
        title_prompt = PromptTemplate(template=self.title_template, input_variables=["user_input","context"])
        llm_chain = LLMChain(prompt=title_prompt, llm=self.llm)
        title_response = llm_chain.run({"user_input": user_input})
        title = title_response.strip().replace("<s>", "").replace("</s>", "").replace("[INST]", "").replace("[/INST]", "").split(":")[-1].strip()
        return title

    def generate_talking_points(self, title):
        # Generate talking points
        tp_prompt = PromptTemplate(template=self.talking_points_template, input_variables=["title","context"])
        llm_chain = LLMChain(prompt=tp_prompt, llm=self.llm)
        tp_response = llm_chain.run({"title": title})
        talking_points = [tp.strip().replace("[INST]", "").replace("[/INST]", "") for tp in tp_response.strip().replace("<s>", "").replace("</s>", "").split('\n') if tp.strip()]
        talking_points = talking_points[:5]
        return talking_points

    def generate_article(self, talking_point):
        # Generate article
        story_prompt = PromptTemplate(template=self.article_template, input_variables=["talking_point","context"])
        llm_chain = LLMChain(prompt=story_prompt, llm=self.llm)
        story_response = llm_chain.run({"talking_point": talking_point})
        story = story_response.strip().replace("<s>", "").replace("</s>", "").replace("[INST]", "").replace("[/INST]", "").split(":")[-1].strip()
        return story

# NEW

In [ ]:
import torch
import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

In [ ]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [ ]:
query = "Explain the difference between ChatGPT and open source LLMs in a couple of lines."
result = llm(
    query
)

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))